In [1]:
import os
import io
from IPython.display import Image, display, HTML
from PIL import Image
import base64 
from transformers import pipeline
import gradio as gr

In [2]:
get_completion = pipeline("ner", 
                          model="dslim/bert-base-NER",
                          token="your token")

def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": output}

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
text = "My name is Behnaz, I'm building DeepLearningAI and I live in Toronto"
get_completion(text)

[{'entity': 'B-PER',
  'score': 0.9991722,
  'index': 4,
  'word': 'Be',
  'start': 11,
  'end': 13},
 {'entity': 'B-PER',
  'score': 0.80218637,
  'index': 5,
  'word': '##hn',
  'start': 13,
  'end': 15},
 {'entity': 'I-PER',
  'score': 0.7582367,
  'index': 6,
  'word': '##az',
  'start': 15,
  'end': 17},
 {'entity': 'B-ORG',
  'score': 0.9912752,
  'index': 12,
  'word': 'Deep',
  'start': 32,
  'end': 36},
 {'entity': 'I-ORG',
  'score': 0.9952278,
  'index': 13,
  'word': '##L',
  'start': 36,
  'end': 37},
 {'entity': 'I-ORG',
  'score': 0.9939958,
  'index': 14,
  'word': '##ear',
  'start': 37,
  'end': 40},
 {'entity': 'I-ORG',
  'score': 0.9946906,
  'index': 15,
  'word': '##ning',
  'start': 40,
  'end': 44},
 {'entity': 'I-ORG',
  'score': 0.8914004,
  'index': 16,
  'word': '##A',
  'start': 44,
  'end': 45},
 {'entity': 'I-ORG',
  'score': 0.5216288,
  'index': 17,
  'word': '##I',
  'start': 45,
  'end': 46},
 {'entity': 'B-LOC',
  'score': 0.9992575,
  'index': 22,
 

In [4]:
def merge_tokens(tokens):
    merged_tokens = []
    for token in tokens:
        if merged_tokens and token['word'][0:2] == '##':#merged_tokens and merged_tokens[-1]['entity'].endswith(token['entity'][2:]):
            # If current token continues the entity of the last one, merge them
            last_token = merged_tokens[-1]
            last_token['word'] += token['word'].replace('##', '')
            last_token['end'] = token['end']
            last_token['score'] = (last_token['score'] + token['score']) / 2
        else:
            # Otherwise, add the token to the list
            merged_tokens.append(token)

    return merged_tokens


def ner(input):
    output = get_completion(input)
    return {"text": input, "entities": merge_tokens(output)}

In [5]:
example_text = "Steve Jobs would be classified as a Person and Apple as an organization."

gr.close_all()
demo = gr.Interface(
    fn=ner,
    inputs=[gr.Textbox(label="Input Text", placeholder="Enter text to find entities", lines=2)],
    outputs=[gr.HighlightedText(label="Text with entities")],
    title="NER with dslim/bert-base-NER",
    description="Find entities using the `dslim/bert-base-NER` model under the hood!",
    #Here are  predefined example for the demo
    examples=[example_text])
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://b85067812f15ca1d51.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
